In [11]:
#%% NumPyの読み込み
import numpy as np
#   Pandasの読み込み
import pandas as pd
#   SciPyのoptimizeモジュールの読み込み
import scipy.optimize as opt
#   NumPyのLinalgモジュールの読み込み
import numpy.linalg as lin

In [12]:
#%% 収益率データの読み込み
R = pd.read_csv('181126_Nakatsuma_report3_data.csv', index_col=0)
T = R.shape[0]
N = R.shape[1]
Mu = R.mean().values
Return_Dev = (R - Mu).values / np.sqrt(T)

In [13]:
#%% リスク寄与度の平準化によるポートフォリオ選択
Stdev = R.std().values
CorrMatrix = np.array([[1.00, 0.25, 0.18, 0.10, 0.25],
                       [0.25, 1.00, 0.36, 0.20, 0.20],
                       [0.18, 0.36, 1.00, 0.25, 0.36],
                       [0.10, 0.20, 0.25, 1.00, 0.45],
                       [0.25, 0.20, 0.36, 0.45, 1.00]])
Sigma = np.diag(Stdev).dot(CorrMatrix).dot(np.diag(Stdev))
iota = np.ones(Mu.shape)
inv_Sigma = lin.inv(Sigma)
Weight_1N = np.tile(1.0/Mu.shape[0], Mu.shape[0])
Weight_MV = inv_Sigma.dot(iota) / iota.dot(inv_Sigma).dot(iota)
Weight_MD = inv_Sigma.dot(Stdev) / iota.dot(inv_Sigma).dot(Stdev)
F = lambda v, Sigma: np.r_[Sigma.dot(v[:-1])-v[-1]/v[:-1], v[:-1].sum()-1.0]
Weight_RP = opt.root(F, np.r_[Weight_1N, 0.0], args=Sigma).x[:-1]
np.set_printoptions(formatter={'float': '{:7.2f}'.format})
print(np.c_[Weight_1N, Weight_MV, Weight_RP, Weight_MD]*100)

[[  20.00   32.04   23.46   28.15]
 [  20.00   14.83   19.20   18.85]
 [  20.00   25.03   20.78   19.12]
 [  20.00   22.34   19.76   22.47]
 [  20.00    5.76   16.80   11.41]]
